In [1]:
from math import sqrt
from itertools import product

import pandas as pd

from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit

In [2]:
supply = pd.read_csv("../data/raw/exisiting_EV_infrastructure_2018.csv")

In [3]:
demand = pd.read_csv("../data/raw/Demand_History.csv")

In [4]:
slow_charger = 200
slow_costs = 1.0*600
fast_charger = 400
fast_costs = 1.5*600

facilities = list(supply[["x_coordinate","y_coordinate"]].itertuples(index=False, name=None))
# real capacities
real_capacities = (supply["existing_num_SCS"]*slow_charger + supply["existing_num_FCS"]*fast_charger).tolist()
# maximum theoretical capacities
capacities = (supply["total_parking_slots"]*fast_charger).tolist()
# maximum slots for chargers
slots = supply["total_parking_slots"].tolist()

slow_slots = supply["existing_num_SCS"].tolist()
fast_slots = supply["existing_num_FCS"].tolist()

## For testing 
# slow_slots = [0]*len(facilities)
# fast_slots = [0]*len(facilities)

customers = list(demand[["x_coordinate","y_coordinate"]].itertuples(index=False, name=None))
demands = demand["2018"].tolist()

In [5]:
sum(capacities) > sum(demands), sum(capacities), sum(demands), sum(real_capacities)

(True, 1000000, 361529.6365968905, 361600)

In [6]:
# This function determines the Euclidean distance between a facility and customer sites.

def compute_distance(loc1, loc2):
    dx = loc1[0] - loc2[0]
    dy = loc1[1] - loc2[1]
    return sqrt(dx*dx + dy*dy)

In [7]:
# Compute key parameters of MIP model formulation

num_facilities = len(facilities)
num_customers = len(customers)
cartesian_prod = list(product(range(num_customers), range(num_facilities)))

In [8]:
# Compute distance matrix

distance = {(c,f): compute_distance(customers[c], facilities[f]) for c, f in cartesian_prod}

In [9]:
# solver = pywraplp.Solver.CreateSolver('CBC_MIXED_INTEGER_PROGRAMMING')
solver = pywraplp.Solver.CreateSolver('SCIP_MIXED_INTEGER_PROGRAMMING')

In [10]:
assign = {}
for i,j in distance.keys(): 
    assign[(i,j)] = solver.NumVar(0,solver.infinity(),"Assign")

In [11]:
# Constraint/Boundaries 2 
slow = {}
for j in range(num_facilities):
    slow[j] = solver.IntVar(slow_slots[j], solver.infinity(), "Slow")

fast = {}
for j in range(num_facilities):
    fast[j] = solver.IntVar(fast_slots[j], solver.infinity(), "Fast")      

In [12]:
# Constraint 3 (slots)
for j in range(num_facilities):
    solver.Add(slow[j] + fast[j] <= slots[j])

In [13]:
# Constraint 5 (capacity constraints)
for j in range(num_facilities):
    solver.Add(sum(assign[(i,j)] for i in range(num_customers)) <= (slow[j]*200 + fast[j]*400))

In [14]:
# Constraint 6 (demand constraints)
for i in range(num_customers):
    solver.Add(sum(assign[(i,j)] for j in range(num_facilities)) == demands[i] )

In [15]:
objective = solver.Objective()

In [16]:
for j in range(num_facilities):
    objective.SetCoefficient(slow[j], slow_costs)
for j in range(num_facilities):
    objective.SetCoefficient(fast[j], fast_costs)    

In [17]:
for i in range(num_customers):
    for j in range(num_facilities):
        objective.SetCoefficient(assign[(i,j)],distance[(i,j)])

In [18]:
objective.SetMinimization()

In [19]:
status = solver.Solve()

In [20]:
if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    print()
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
else:
    print('The problem does not have an optimal solution.')

Objective value = 2276576.673923479

Problem solved in 19944.000000 milliseconds
Problem solved in 14110 iterations
Problem solved in 1 branch-and-bound nodes


In [21]:
# display optimal values of decision variables
for j in range(num_facilities):
    print(f"\n Build {slow[j].solution_value() - slow_slots[j]} slow charger  and {fast[j].solution_value() - fast_slots[j]} fast charger at location {j + 1}.")


 Build 0.0 slow charger  and 6.0 fast charger at location 1.

 Build 0.0 slow charger  and 0.0 fast charger at location 2.

 Build 0.0 slow charger  and 0.0 fast charger at location 3.

 Build 0.0 slow charger  and 0.0 fast charger at location 4.

 Build 0.0 slow charger  and 0.0 fast charger at location 5.

 Build 0.0 slow charger  and 0.0 fast charger at location 6.

 Build 0.0 slow charger  and 0.0 fast charger at location 7.

 Build 0.0 slow charger  and 0.0 fast charger at location 8.

 Build 0.0 slow charger  and 0.0 fast charger at location 9.

 Build 0.0 slow charger  and 0.0 fast charger at location 10.

 Build 0.0 slow charger  and 8.0 fast charger at location 11.

 Build 0.0 slow charger  and 0.0 fast charger at location 12.

 Build 0.0 slow charger  and 8.0 fast charger at location 13.

 Build 0.0 slow charger  and 0.0 fast charger at location 14.

 Build 0.0 slow charger  and 0.0 fast charger at location 15.

 Build 0.0 slow charger  and 5.0 fast charger at location 16.



In [22]:
# Shipments from facilities to customers.

for i, j in assign.keys():
    if (abs(assign[i, j].solution_value()) > 1e-6):
        print(f"\n Demand point {i + 1} receives {round(assign[i, j].solution_value(), 2)} of its demand {round(demands[i],2)} from parking slot {j + 1} .")


 Demand point 1 receives 13.12 of its demand 13.12 from parking slot 39 .

 Demand point 2 receives 12.02 of its demand 12.02 from parking slot 39 .

 Demand point 3 receives 14.02 of its demand 14.02 from parking slot 39 .

 Demand point 4 receives 15.01 of its demand 15.01 from parking slot 39 .

 Demand point 5 receives 16.36 of its demand 16.36 from parking slot 39 .

 Demand point 6 receives 17.56 of its demand 17.56 from parking slot 39 .

 Demand point 7 receives 19.7 of its demand 19.7 from parking slot 39 .

 Demand point 8 receives 19.37 of its demand 19.37 from parking slot 39 .

 Demand point 9 receives 21.9 of its demand 21.9 from parking slot 39 .

 Demand point 10 receives 21.77 of its demand 21.77 from parking slot 39 .

 Demand point 11 receives 22.6 of its demand 22.6 from parking slot 39 .

 Demand point 12 receives 23.92 of its demand 23.92 from parking slot 39 .

 Demand point 13 receives 19.59 of its demand 19.59 from parking slot 25 .

 Demand point 14 receives 

In [23]:
total_supply = 0
for j in range(num_facilities):
    total_supply += slow[j].solution_value()*200 + fast[j].solution_value()*400

In [24]:
sum(demands), total_supply 

(361529.6365968905, 429600.0)

In [25]:
total_demand = 0.0
for i in range(num_customers):
    for j in range(num_facilities):
        total_demand += assign[(i,j)].solution_value()

In [26]:
sum(demands), total_demand

(361529.6365968905, 361529.6365968906)